In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
submit1 = pd.read_csv('submit-0.856.csv')
submit2 = pd.read_csv('submit-1.262.csv')

In [8]:
submit2

,contest-tmp2m-14d__tmp2m,index
0,29.551507,375734
1,29.498855,375735
2,29.521992,375736
3,29.959085,375737
4,29.899663,375738
...,...,...
31349,2.996932,407083
31350,3.085458,407084
31351,2.346873,407085
31352,3.289782,407086


In [10]:
submit2 = submit2.rename(columns={'contest-tmp2m-14d__tmp2m': 'contest-tmp2m-14d__tmp2m_2',
                'index': 'index_2'})
submit_total = pd.concat([submit1, submit2], axis=1)
submit_total

,contest-tmp2m-14d__tmp2m,index,contest-tmp2m-14d__tmp2m_2,index_2
0,29.540273,375734,29.551507,375734
1,29.486998,375735,29.498855,375735
2,29.227439,375736,29.521992,375736
3,29.370673,375737,29.959085,375737
4,29.553691,375738,29.899663,375738
...,...,...,...,...
31349,5.644368,407083,2.996932,407083
31350,5.911460,407084,3.085458,407084
31351,5.915983,407085,2.346873,407085
31352,4.959525,407086,3.289782,407086


In [14]:
submit_total['diff'] = abs(submit_total['contest-tmp2m-14d__tmp2m'] - submit_total['contest-tmp2m-14d__tmp2m_2'])
submit_total.sort_values('diff', ascending=False)

,contest-tmp2m-14d__tmp2m,index,contest-tmp2m-14d__tmp2m_2,index_2,diff
27937,10.961280,403671,5.150804,403671,5.810476
26229,11.733764,401963,5.968319,401963,5.765445
26290,10.930837,402024,5.192491,402024,5.738347
24643,12.130153,400377,6.644503,400377,5.485650
26168,11.241171,401902,5.921467,401902,5.319704
...,...,...,...,...,...
17531,13.201723,393265,13.201540,393265,0.000183
22351,9.842597,398085,9.842749,398085,0.000152
20887,12.269757,396621,12.269618,396621,0.000139
28488,15.101028,404222,15.101035,404222,0.000007


In [15]:
# df_test = pd.read_csv('data/test_data.csv')

## Plot distribution of train set with month 12

In [19]:
from sklearn.preprocessing import LabelEncoder

In [46]:
df_train = pd.read_csv('data/train_data.csv')
df_test = pd.read_csv('data/test_data.csv')

In [47]:
def fill_nulls(df):

    tmp = pd.DataFrame(df.isna().sum()) \
        .rename(columns={0: 'cnt'})

    null_ls = tmp[tmp.cnt != 0].reset_index()

    mean_values = {}

    for i in null_ls['index']:
        mean = df.loc[:, i].mean()
        mean_values[i] = mean

    for k, v in mean_values.items():
        df[k].fillna(value=v, inplace=True)

    return df

df_train = fill_nulls(df_train)

In [48]:
def label_encoding(df_train, df_test):
    le = LabelEncoder()

    le.fit(df_train['climateregions__climateregion'])

    df_train['climateregions__climateregion'] = le.transform(df_train['climateregions__climateregion'])
    df_test['climateregions__climateregion'] = le.transform(df_test['climateregions__climateregion'])

    return df_train, df_test

df_train, df_test = label_encoding(df_train, df_test)

In [49]:
column_date="startdate"

def get_idx(lat, lon):
    return str(round(lat, 4)) + "_" + str(round(lon, 4))

def get_date(date, idx=0):
    return int(date.split('/')[idx])

def create_cols(df):
    df["idx"] = np.vectorize(get_idx)(df['lat'], df['lon'])
    df["month"] = np.vectorize(get_date)(df[column_date], idx=0)
    # df["day"] = np.vectorize(get_date)(df[column_date], idx=1)
    df["year"] = np.vectorize(get_date)(df[column_date], idx=2)

    return df

df_train = create_cols(df_train)
df_test = create_cols(df_test)

In [50]:
df_train.head()

,index,lat,lon,startdate,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,...,wind-vwnd-925-2010-14,wind-vwnd-925-2010-15,wind-vwnd-925-2010-16,wind-vwnd-925-2010-17,wind-vwnd-925-2010-18,wind-vwnd-925-2010-19,wind-vwnd-925-2010-20,idx,month,year
0,0,0.0,0.833333,9/1/14,237.00,29.02,31.64,29.57,30.73,29.71,...,9.56,-2.03,48.13,28.09,-13.50,11.90,4.58,0.0_0.8333,9,14
1,1,0.0,0.833333,9/2/14,228.90,29.02,31.64,29.57,30.73,29.71,...,21.17,4.44,48.60,27.41,-23.77,15.44,3.42,0.0_0.8333,9,14
2,2,0.0,0.833333,9/3/14,220.69,29.02,31.64,29.57,30.73,29.71,...,32.16,5.01,48.53,19.21,-33.16,15.11,4.82,0.0_0.8333,9,14
3,3,0.0,0.833333,9/4/14,225.28,29.02,31.64,29.57,30.73,29.71,...,39.66,-1.41,50.59,8.29,-37.22,18.24,9.74,0.0_0.8333,9,14
4,4,0.0,0.833333,9/5/14,237.24,29.02,31.64,29.57,30.73,29.71,...,38.62,-5.21,54.73,-2.58,-42.30,21.91,10.95,0.0_0.8333,9,14


In [51]:
drop_col = pd.read_csv("data/correlations_with_target_greater_0.7.csv")
drop_col = drop_col["col"].values
drop_col = [each for each in drop_col if "contest" not in each and "wind" not in each]

In [52]:
df_train.drop(columns=drop_col, inplace=True)
df_test.drop(columns=drop_col, inplace=True)

In [53]:
val = df_train[(df_train.month == 12) & (df_train.year == 2015)]
train = df_train[~((df_train.month == 12) & (df_train.year == 2015))]